**TIP:** This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```
Looks like this, it will click the screen every 10 minutes so that you don't get kicked off for being idle! HACKS!

# Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 10501, done.
remote: Total 10501 (delta 0), reused 0 (delta 0), pack-reused 10501
Receiving objects: 100% (10501/10501), 10.72 MiB | 21.96 MiB/s, done.
Resolving deltas: 100% (7253/7253), done.
/content/yolov5
     |████████████████████████████████| 596 kB 11.9 MB/s 


In [ ]:
import os
from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
import torch

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.10.0+cu111 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


# Import Data

In [ ]:
!pip install roboflow

     |████████████████████████████████| 145 kB 12.8 MB/s 
     |████████████████████████████████| 178 kB 50.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.3 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 138 kB 53.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.1-py3-none-any.whl size=20916 sha256=60970a08510eccb323ac710cb29479abd3877264ad1e1cf490329b76a3dd7fda
  Stored in directory: /root/.cache/pip/wheels/57/00/03/2d83f3e90ff41c36eef9c3747c328290c01b06e2619f9ed7b6
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=a0b624d3b666fc7fb81ebaa81af4daa6027fad6688f7cbf190bbd19097f9078e
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="############")
project = rf.workspace().project("finger_tips-to48f")
dataset = project.version(5).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Finger_tips-5 in yolov5pytorch:: 100%|██████████| 25675/25675 [00:22<00:00, 1130.24it/s]


# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Finger_tip/backup/exp3/weights/last.pt /content/last.pt   #weights path to continue traning

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance  #--batch 16
%%time
%cd /content/yolov5/      
!python train.py --img 416 --epochs 500 --data /content/yolov5/Finger_tips-5/data.yaml --weights /content/last.pt --cache --project /content/drive/MyDrive/Finger_tip/backup/ # path to save backup

/content/yolov5
train: weights=/content/last.pt, cfg=, data=/content/yolov5/Finger_tips-5/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=500, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/Finger_tip/backup/, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-187-gf3085ac torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gam

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

## path to save weights on Gdrive

In [ ]:
!cp /content/yolov5/runs/train/exp/weights/best.pt /content/drive/MyDrive/Finger_tip/07.best.pt
!cp /content/yolov5/runs/train/exp/weights/last.pt /content/drive/MyDrive/Finger_tip/07.last.pt

## Download Weights

In [ ]:
from google.colab import files

files.download('/content/yolov5/runs/train/exp/weights/best.pt')